<a href="https://colab.research.google.com/github/fbsaif007/fbsaif007/blob/main/Epicured_NumberofSubscribers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Token Generator

In [ ]:
# Obtain a token for the Prado API

import requests
import json

# Define the API endpoint and the payload
url = "https://api.getprado.com/api/v2/Auth/obtainToken"
payload = {
    "apiKey": "**************",
    "secret": "*************"
}

# Send a POST request to the API to obtain the token
response = requests.post(url, json=payload)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Extract the token
    token = data.get("token")

    if token:
        # Save the token to a text file
        with open('token.txt', 'w') as file:
            file.write(token)
        print("Token successfully obtained and stored in token.txt")
    else:
        print("Token not found in the response")
else:
    print(f"Failed to obtain token. Status code: {response.status_code}")
    print("Response:", response.text)

Token successfully obtained and stored in token.txt


## Data Extraction from Prado APIs



# Data Extraction, transformation and cleaning for find the number of subscribers and potential new orders

In [ ]:
import sys
import requests
import pandas as pd
import numpy as np
import json
import csv
from datetime import datetime, timedelta
import time
from copy import deepcopy

# Debug
debug = 'false'

def get_start_and_end_date(current_date=None):
    if current_date:
        current_date = datetime.strptime(current_date, "%Y-%m-%d")
    else:
        current_date = datetime.today()

    day_of_week = current_date.weekday()

    if day_of_week >= 4:  # Friday (4), Saturday (5), Sunday (6)
        # Start date is the Monday following the next Monday
        start_date = current_date + timedelta(days=(7 - day_of_week) + 7)
        # End date is the Friday following the next Friday
        end_date = start_date + timedelta(days=4)
    else:  # Monday (0) to Thursday (3)
        # Start date is the next Monday
        start_date = current_date + timedelta(days=(7 - day_of_week))
        # End date is the immediate next Friday after the start date
        end_date = start_date + timedelta(days=4)
    return start_date.date(), end_date.date()

def main():
    # Instead of sys.argv, allow setting the override date directly or prompt for input
    override_date = input("Enter an override date (YYYY-MM-DD) or leave blank for today's date: ")
    override_date = override_date if override_date.strip() else None

    start, end = get_start_and_end_date(override_date)
    print(f"Start Date: {start}, End Date: {end}")

    # Read the token from a file
    with open('token.txt', 'r') as file:
        token = file.read().strip()

    # Base URL
    baseurl = 'https://api.getprado-release.com/api/v2/' if debug == 'true' else 'https://api.getprado.com/api/v2/'

    # Headers
    headers = {'Authorization': f'Bearer {token}'}

    # Initial Limit and Offset values
    limit = 100
    offset = 0
    rate_limit_delay = 0.6  # Rate limit

    # Breakout functions
    def cross_join(left, right):
        new_rows = [] if right else left
        for left_row in left:
            for right_row in right:
                temp_row = deepcopy(left_row)
                for key, value in right_row.items():
                    temp_row[key] = value
                new_rows.append(deepcopy(temp_row))
        return new_rows


    def flatten_list(data):
        for elem in data:
            if isinstance(elem, list):
                yield from flatten_list(elem)
            else:
                yield elem


    def json_to_dataframe(data_in):
        def flatten_json(data, prev_heading=''):
            if isinstance(data, dict):
                rows = [{}]
                for key, value in data.items():
                    rows = cross_join(rows, flatten_json(value, prev_heading + '.' + key))
            elif isinstance(data, list):
                rows = []
                for item in data:
                    [rows.append(elem) for elem in flatten_list(flatten_json(item, prev_heading))]
            else:
                rows = [{prev_heading[1:]: data}]
            return rows

        return pd.DataFrame(flatten_json(data_in))

    ######
    # Order loop
    ######

    # Orders API endpoint and parameters
    orders_endpoint = "ActiveOrders/gather/"
    orders_params = {
        'fulfillmentDateStart': str(start),
        'fulfillmentDateEnd': str(end)
    }

    orders = []

    # API loop to fetch all data
    while True:
        print("-----")
        print("Downloading orders...")
        print("-----")

        # Construct the URL
        orders_url = f"{baseurl}{orders_endpoint}?offset={offset}&limit={limit}"
        print("Requesting", orders_url)

        # API call
        response = requests.get(orders_url, headers=headers, params=orders_params)

        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break  # Stop on error

        orders_data = response.json()

        # If no data, stop
        if not orders_data:
            print("No more data available.")
            break

        orders.extend(orders_data)

        # Print first order's email as a debug step
        try:
            print("First order email:", orders_data[0]['email'])
        except IndexError:
            print("No more orders.")
            break

        offset += limit  # Increment the offset for pagination
        time.sleep(rate_limit_delay)  # Respect the rate limit

    total_orders = len(orders)
    print(f"Total orders fetched: {total_orders}")

    # Convert the orders to a DataFrame
    orders_df = pd.DataFrame(orders)
    print("Flattening and exporting results.")

    # Save full orders as CSV
    orders_df.to_csv(f"closedOrders-all-cols-{datetime.today().date()}.csv", index=False)

    # Select specific columns for a more compact CSV
    cols = [
        "customerId", "id", "number", "customerFullName", "email", "itemsCount",
        "fulfillmentDate", "fulfillmentStatus", "deliveryAddressFull", "totalAmount"
    ]
    orders_df[cols].to_csv(f"closedOrders-selected-cols-{datetime.today().date()}.csv", index=False)

    print("CSV files have been created.")

    ######
    # Subscriptions loop
    ######

    offset = 0

    # Subscriptions API endpoint and parameters
    subscriptions_endpoint = "ActiveSubscriptionInstances/gather/"
    subscriptions_params = {
        'fulfillmentDateStart': str(start),
        'fulfillmentDateEnd': str(end),
        'includeSubscription': 'true'
    }

    subscriptions = []

    # API loop to fetch all data
    while True:
        print("-----")
        print("Downloading subscriptions...")
        print("-----")

        # Construct the URL
        subscriptions_url = f"{baseurl}{subscriptions_endpoint}?offset={offset}&limit={limit}"
        print("Requesting", subscriptions_url)

        # API call
        response = requests.get(subscriptions_url, headers=headers, params=subscriptions_params)

        if response.status_code != 200:
            print("Error:", response.status_code, response.text)
            break  # Stop on error

        subscriptions_data = response.json()

        # If no data, stop
        if not subscriptions_data:
            print("No more data available.")
            break

        subscriptions.extend(subscriptions_data)

        # Print first order's email as a debug step
        try:
            print("First subscription id:", subscriptions_data[0]['subscriptionId'])
        except IndexError:
            print("No more subscriptions.")
            break

        offset += limit  # Increment the offset for pagination
        time.sleep(rate_limit_delay)  # Respect the rate limit

    total_subscriptions = len(subscriptions)
    print(f"Total subscriptions fetched: {total_subscriptions}")

    # Convert the orders to a DataFrame
    subscriptions_df = json_to_dataframe(subscriptions)
    print("Flattening and exporting results.")

    print("------------------------------")

    # Save full orders as CSV
    subscriptions_df.to_csv(f"subccriptions-all-cols-{datetime.today().date()}.csv", index=False)

    print(f"Week Starting on: {str(start)} to: {str(end)}")

    paid_orders = orders_df[orders_df['totalAmount'] > 0]
    print(f"New Paid Orders: {len(paid_orders)} of {total_orders} total orders")

    zero_dollar_orders = orders_df[orders_df['totalAmount'] == 0]
    print(f"Zero Dollar Orders: {len(zero_dollar_orders)}")

    # Sum of orders.itemsCount
    total_orders_items_count = orders_df['itemsCount'].sum()
    print(f"Total new order SKUs: {total_orders_items_count}")

    valid_subscriptions = subscriptions_df[
    (subscriptions_df['subscription.isPaused'] == False) &
    (subscriptions_df['subscription.itemsCount'] >= 1)
    ]
    print(f"Valid Subscriptions: {len(valid_subscriptions)} of {total_subscriptions} total subscriptions")

    # Sum of subscription.itemsCount
    total_items_count = valid_subscriptions['subscription.itemsCount'].sum()
    print(f"Total valid subscription SKUs: {total_items_count}")

    # Calculate the combined total exepcted orders
    combined_total_expected_orders = len(paid_orders) + len(valid_subscriptions)
    print(f"Combined total expected orders (valid subscriptions + new paid orders): {combined_total_expected_orders}")

    # Calculate the combined total itemsCount/SKUs
    combined_total_items = total_items_count + total_orders_items_count
    print(f"Combined total paid SKUs (subscriptions + orders): {combined_total_items}")

    # Count valid subscriptions based on subscription.frequency
    weekly_subscriptions = valid_subscriptions[valid_subscriptions['subscription.frequency'] == 0]
    biweekly_subscriptions = valid_subscriptions[valid_subscriptions['subscription.frequency'] == 1]
    monthly_subscriptions = valid_subscriptions[valid_subscriptions['subscription.frequency'] == 2]

    # Print the counts
    print(f"Number of weekly subscriptions (frequency = 0): {len(weekly_subscriptions)}")
    print(f"Number of biweekly subscriptions (frequency = 1): {len(biweekly_subscriptions)}")
    print(f"Number of monthly subscriptions (frequency = 2): {len(monthly_subscriptions)}")

if __name__ == "__main__":
    main()

Enter an override date (YYYY-MM-DD) or leave blank for today's date: 
Start Date: 2025-01-13, End Date: 2025-01-17
-----
-----
Requesting https://api.getprado.com/api/v2/ActiveOrders/gather/?offset=0&limit=100
First order email: katherinebaigrie@hotmail.com
-----
-----
Requesting https://api.getprado.com/api/v2/ActiveOrders/gather/?offset=100&limit=100
No more data available.
Total orders fetched: 54
Flattening and exporting results.
CSV files have been created.
-----
-----
Requesting https://api.getprado.com/api/v2/ActiveSubscriptionInstances/gather/?offset=0&limit=100
First subscription id: cca86aea-08ee-4c36-b134-08db8a68603d
-----
-----
Requesting https://api.getprado.com/api/v2/ActiveSubscriptionInstances/gather/?offset=100&limit=100
First subscription id: edcf8f1f-e959-492f-c6ec-08db8a685075
-----
-----
Requesting https://api.getprado.com/api/v2/ActiveSubscriptionInstances/gather/?offset=200&limit=100
First subscription id: dac855b3-936b-4abd-2279-08dc9d69ed0f
-----
-----
Request